# Combine Trajectories

The cell below concatenates two molecular trajectories A and B that were started from the same initial state. Trajectory A is a glycine crystal that dissolves. Trajectory B is a glycine crystal that grows. Before these trajectories are merged, A is inverted. This results in a long trajectory that crosses the nucleation barrier.

In [1]:
import MDAnalysis as mda
import os

# Define topology (shared by trajectories)
topology_file = f'Trajectories/1.7nm/gamma_glycine_crystal_3_2_3_box_5.0_1.7nm_sphere_insert_303_solv_minim_npteq275K.gro'

# Define trajectories file names
trajectory_A = 'Trajectories/1.7nm/gamma_glycine_crystal_3_2_3_box_5.0_1.7nm_sphere_insert_303_solv_minim_npteq275K_md_run8_short.trr' 
trajectory_B = 'Trajectories/1.7nm/gamma_glycine_crystal_3_2_3_box_5.0_1.7nm_sphere_insert_303_solv_minim_npteq275K_md_run3_short.trr'  

# Load the topology A
print("Loading topology for Run A...")
traj_A = mda.Universe(topology_file, trajectory_A)
print("Topology loaded.")

# Reverse A and write to a new trajectory (positions and velocities)
with mda.Writer('reversed_A.trr', n_atoms=traj_A.atoms.n_atoms) as W:
    print("Reversing Run A frames...")
    for ts in reversed(traj_A.trajectory[1:]):  # First frame in common with B is skipped
        # Reverse velocities as well if available
        if hasattr(traj_A.trajectory, 'velocities'):
            traj_A.atoms.velocities *= -1  # Invert the velocities
        W.write(traj_A.atoms)

# Load reversed trajectory A
print("Loading topology reversed A...")
u_reversed = mda.Universe(topology_file, 'reversed_A.trr')
print("Topology loaded.")

# Load the topology for Run B
print("Loading topology for Run B...")
traj_B = mda.Universe(topology_file, trajectory_B)
print("Topology loaded.")

# Concatenate trimmed reversed Run A with Run B
print("Concatenating trajectories...")
with mda.Writer('trajectory_A_B.trr', n_atoms=traj_A.atoms.n_atoms) as W:
    # Write frames from reversed A
    for ts in u_reversed.trajectory:
        W.write(u_reversed.atoms)
    # Write frames from B
    for ts in traj_B.trajectory:
        W.write(traj_B.atoms)

print(f"Combined trajectory saved successfully as trajectory_A_B.trr")


Loading topology for Run A...
Topology loaded.
Reversing Run A frames...
Loading topology reversed A...
Topology loaded.
Loading topology for Run B...
Topology loaded.
Concatenating trajectories...
Combined trajectory saved successfully as trajectory_A_B.trr
